## Limpieza de la data

#### Primero vamos a elmininar la copia.db

In [2]:
import os

# Nombre del archivo de la base de datos
db_file = "../data/processed/copia.db"

# Verificar si existe y eliminarlo
if os.path.exists(db_file):
    os.remove(db_file)
    print("Base de datos eliminada exitosamente.")
else:
    print("El archivo de la base de datos no existe.")

El archivo de la base de datos no existe.


In [3]:
import pandas as pd
import sqlite3

In [5]:
# Cargar datos
df = pd.read_csv("../data/raw/ufo_data.csv")
print("✅ Datos cargados")

✅ Datos cargados


C:\Users\harri\AppData\Local\Temp\ipykernel_1348\2705268744.py:2: DtypeWarning: Columns (5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/raw/ufo_data.csv")


In [6]:
# Crearemos una conexión sqlite3
conexion = sqlite3.connect("../data/raw/ufo_data.db")

# Guardar el dataframe como tabla SQLite
df.to_sql("ufo_table", conexion, if_exists="replace", index=False) 
# El index col se usa para eliminar el Unnamed

print("✅ Base de datos creada correctamente...")

cursor = conexion.cursor()

✅ Base de datos creada correctamente...


In [7]:
# Crear una copia de la db
copia = "../data/processed/copia.db"

consulta = f"""
ATTACH DATABASE '{copia}' AS copia;
SELECT sql FROM sqlite_master WHERE type='table';
"""
cursor.executescript(consulta)

# Copiar las tablas a la base de datos de respaldo
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tablas = cursor.fetchall()

for tabla in tablas:
    nombre_tabla = tabla[0]
    consulta_copia = f"CREATE TABLE copia.{nombre_tabla} AS SELECT * FROM {nombre_tabla};"
    cursor.execute(consulta_copia)

print("✅ Copia de la BD creada correctamente...")
# Ya no volver a ejecutar esto
# Si lo hacemos primero hay que cerrar la conexión
# Y luego hay que eliminar el archivo copia.db

✅ Copia de la BD creada correctamente...


Lo que haremos en esta fase:

2. Ahora si procedemos hacer la limpieza:

    ✅ Datos faltantes (Nulos).
    
    ✅ Registros duplicados.
    
    ✅ Formatos inconsistentes (fechas, nombres, números).
    
    ✅ Valores atípicos (outliers).

### ✅ Manejar Datos Faltantes (Nulos)

In [8]:
# Reemplazar Nulos
consulta = """
           UPDATE copia.ufo_table
           SET
               state = COALESCE(state, "Desconocido"),
               country = COALESCE(country, "Desconocido"),
               shape = COALESCE(shape, "No especificado"),
               comments = COALESCE(comments, "Sin comentarios")
           """
conexion.execute(consulta)
conexion.commit()
print("Valores reemplazados con éxito")

Valores reemplazados con éxito


In [9]:
nombre_columnas = "PRAGMA table_info(ufo_table);"
nombre_columnas = pd.read_sql_query(nombre_columnas, conexion)
nombre_columnas

,cid,name,type,notnull,dflt_value,pk
0,0,datetime,TEXT,0,None,0
1,1,city,TEXT,0,None,0
2,2,state,TEXT,0,None,0
3,3,country,TEXT,0,None,0
4,4,shape,TEXT,0,None,0
5,5,duration (seconds),TEXT,0,None,0
6,6,duration (hours/min),TEXT,0,None,0
7,7,comments,TEXT,0,None,0
8,8,date posted,TEXT,0,None,0
9,9,latitude,TEXT,0,None,0


### ✅ Manejar registros duplicados

In [10]:
# Eliminar duplicados
eliminar = """
           WITH cte AS (
                SELECT ROWID, ROW_NUMBER() OVER(PARTITION BY datetime, city, state, country, shape) AS num_fila
                FROM copia.ufo_table
           )
           DELETE FROM copia.ufo_table
           WHERE ROWID IN (
               SELECT ROWID FROM cte WHERE num_fila > 1
           );
           """
conexion.execute(eliminar)
conexion.commit()
print("Duplicados eliminados con éxito")

Duplicados eliminados con éxito


### ✅ Normalizar los tipos de datos (Fechas, numeros)

In [ ]:
# Convertir datetime de TEXT a DATETIME(ya que es fecha y hora, si fuera solo fecha sería DATE y si solo fuera hora sería TIME)
# Convertir duration seconds de TEXT a REAL
formatear = """
            UPDATE copia.ufo_table
            SET `duration (seconds)` = CAST(`duration (seconds)` AS REAL)
            ;
            """
conexion.execute(formatear)
conexion.commit()
print("Datos normalizados con éxito")

#### Nos quedamos acá:
- Falta Formatear y aplicar Identificación de Ouliers para `duration (seconds)`

In [ ]:
# Cerrar conexion
#conexion.close()